In [ ]:
# Copyright (c) Meta Platforms, Inc. and affiliates.

# Extract ImageNet embeddings with DINOv2

In [ ]:
import os
from pathlib import Path
import torch
from torch.utils.data import DataLoader
from dinov2.hub import dinov2_vitb14
from dinov2.data.datasets import ImageNet
from dinov2.data.transforms import make_classification_eval_transform


In [ ]:
imagenet_root = '<FIXME>'  # Path to ImageNet root
extra_root = imagenet_root
output_dir = Path('features_output')
output_dir.mkdir(parents=True, exist_ok=True)

transform = make_classification_eval_transform()
dataset = ImageNet(split=ImageNet.Split.VAL, root=imagenet_root, extra=extra_root, transform=transform)
dataloader = DataLoader(dataset, batch_size=64, num_workers=4, shuffle=False)


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = dinov2_vitb14(pretrained=True)
model.to(device)
model.eval()


In [ ]:
features = []
labels = []
with torch.no_grad():
    for imgs, target in dataloader:
        imgs = imgs.to(device)
        feats = model(imgs)
        features.append(feats.cpu())
        labels.append(target)
features = torch.cat(features)
labels = torch.cat(labels)
torch.save({'features': features, 'labels': labels}, output_dir / 'imagenet_features.pt')
